In [8]:
import numpy as np
import pickle
import pandas as pd

# G-code generation codes are cloned from https://github.com/tibor-barsi/GcodeGenerator. Tibor Barsi is the author of the code, and was a PhD student at the Ladisk lab of the University of Ljubljana
#  We should be careful with crediting the author of the code, if we ever want to make these code public.
from src.g_code_generation_copy.gcode_generator import G_code_generator
from src.g_code_generation_copy.tool_changer_functions import save_params, load_params, printer_start, load_tool, unload_tool, tool_change, take_photo, play_sound, printer_stop
# from src.additional_functions import *
from src.network import Network_custom, replace_brackets
import os

from src.src.uw_spider.fd import FD_Network

BYU_UW_root = r"G:\.shortcut-targets-by-id\1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx\Illimited Lab Projects\Research Projects\Spiders\BYU-UW"

import plotly.io as pio
pio.renderers.default = 'browser'


eccentricity = $(\textnormal{upper}-\textnormal{lower})/(\textnormal{upper}+\textnormal{lower})$
narrowness = $\sqrt{1-(L_{\text{hor}}/L_{\text{vert}})^2}$, set negative if $L_{\text{hor}} > L_{\text{vert}}$

In [9]:
Heigths = np.array([76+75, 77.3+82, 80+85])
Widths = np.array([75+75.6, 68+71.7, 62.8+64.3])
tops = np.array([76+1.6, 77.3-12.1, 80-19])
bottoms = np.array([75+-1.6, 82+12.1, 85+19])

ecc = (tops-bottoms)/(tops+bottoms)
nar = np.sqrt(1 - (Widths/Heigths)**2)
ecc, nar, Widths>Heigths

(array([ 0.02781457, -0.18141871, -0.26060606]),
 array([0.07273923, 0.48056026, 0.63767801]),
 array([False, False, False]))

In [24]:
# model_name = "Web_spirals_r56_s19"
model_name = "Web_constant_length"
ecc = 1
gap = 0
spider_root = r"C:\Users\thijs\Documents\GitHub\uw_spider"
spider_path = os.path.join(spider_root, 'data', 'networks', "Constraint", model_name, model_name + f"_ecc{ecc}_gap{gap}_constraint.json")

net = FD_Network.from_json(spider_path)
net.plotly_net_plot()

net_copy = Network_custom.direct(np.array(net.vertex_list()), net.edges_list(), net.q_list(), net.fixed)

Let's see if the optimization works for this case

In [4]:
file_path = os.path.join(BYU_UW_root, 'Tensile Testing', 'Avg_Stress_Strain_Overture_TPU_1mm2.csv')
stress_data, strain_data = net_copy.load_stress_strain_curve(file_path, A_scale = 1.0)

TPU_nl = {'stress':strain_data, 'strain': stress_data, 'v':0.3897, 'p':1.18e-9, 'A': 0.078294515, 'name': 'TPU Overture'} # TPU Overture non-conductive

A = [TPU_nl['A']]*len(net.edges)
l0, l_scalar = net_copy.materialize_nonlinear(A, stress_data, strain_data, interpolation_kind = 'cubic')
# Yielding the initial lengths of each element. And the following scalar: min(l0/l1)
# net.l0, net.l_scalar


correction_scalar = 1
net_copy.initialize_shape_optimizer(function_type = 'standard',  method = 'Gauss-Seidel',options ={"maxiter": 100000, "damping": .1, 'correction_scalar': correction_scalar})
# net_copy.initialize_shape_optimizer(function_type = "no optimization")
net_copy.optimize_vertices()

Iteration 0: Current error = 4.646419899372911
Iteration 100: Current error = 0.5189640110575123
Iteration 200: Current error = 0.3618222028904645
Iteration 300: Current error = 0.3038240329648949
Iteration 400: Current error = 0.2707109014601424
Iteration 500: Current error = 0.24793686506496956
Iteration 600: Current error = 0.2306561204815458
Iteration 700: Current error = 0.21684515484768296
Iteration 800: Current error = 0.20530913777538687
Iteration 900: Current error = 0.1955589412372597
Iteration 1000: Current error = 0.18704142434178173


KeyboardInterrupt: 

In [ ]:
%matplotlib qt
net_copy.net_plot()
import matplotlib.animation as animation
ani = net_copy.ShapeOptimizer.animate_optimization()
writer = animation.FFMpegWriter(fps=15)
ani.save(os.path.join(BYU_UW_root, 'images', 'form finding animations', f'form_finding_{model_name}_{0}.mp4'), writer=writer)

In [11]:
print(l_scalar)
reference_point = [0,0,0] 
net_copy.scale_vertices(reference_point, net_copy.l_scalar, account_for_leafs=False) # If you don't provide a scalar, it will use network.l_scalar automatically. Note, that in this particular case, there are no leafs, so the last argument is not necessary.

#  You can plot the scaled network like this
# net.net_plot(color=True, plot_type='scaled')

# Determine the Radius and Angle of the circle that define the arc of each element
R, th = net_copy.arc_param()
# for the arc length (net.l1 * net.l_scalar) is used unless specified otherwise, for the cord length (net.l0) is used unless specified otherwise
xyz = net_copy.arc_points(n = 9) 

net_copy.net_plot(color=True, plot_type='arcs', elables=False)
net_copy.l_scalar, 1/np.min(R)

0.9684234284513996


AttributeError: 'Network_custom' object has no attribute 'l_scalar'

In [6]:
net.edges_list()[50], net.edges[50]

([49, 14], [49, 14])

In [12]:
# Get the list of directions in the order of net.vertices keys
directions_ordered = [net.direction[key] for key in net.edges.keys()]

# Get the list of directions in the order of sorted keys
sorted_keys = sorted(net.edges.keys())
directions_ordered2 = [net.direction[key] for key in sorted_keys]

net_copy.dir = directions_ordered2
directions_ordered[:10], directions_ordered2[:10]

AttributeError: 'FD_Network' object has no attribute 'direction'

In [25]:
import matplotlib.pyplot as plt
%matplotlib qt
tex_lines = [r"\begin{tikzpicture}"]

# Compute min and max force values without storing them in a list
f_min, f_max = float("inf"), float("-inf")
for key in net.edges:
    f = net.f[key]
    if f < f_min:
        f_min = f
    if f > f_max:
        f_max = f

cmap = plt.get_cmap("viridis")  # Choose a colormap

gamma = 3  # Adjust this value to control the sensitivity to smaller values
fig, ax = plt.subplots(figsize=(7, 7), dpi=100)
ax.set_aspect('equal')
ax.axis('off')
plt.tight_layout()
for key, edge in net.edges.items():
    v0, v1 = edge
    f = net.f[key]
    coor0, coor1 = net.vertex_xyz(v0), net.vertex_xyz(v1)

    # Normalize force
    norm_f = (f - f_min) / (f_max - f_min) if f_max > f_min else 0.5

    # Apply gamma correction
    norm_f = norm_f**gamma  

    # Rescale back to [0,1] using Min-Max scaling
    min_gamma = f_min**gamma if f_min > 0 else 0  # Avoid issues with negative forces
    max_gamma = f_max**gamma if f_max > 0 else 1

    norm_f = (norm_f - min_gamma) / (max_gamma - min_gamma) if max_gamma > min_gamma else 0.5


    # Get RGB color from colormap
    r, g, b, _ = cmap(norm_f)

    tex_lines.append(fr"""
    \definecolor{{edge{key}}}{{rgb}}{{{r:.3f},{g:.3f},{b:.3f}}}
    \draw[color=edge{key}] 
        ({coor0[0]/10:.3f}, {coor0[1]/10:.3f}) -- 
        ({coor1[0]/10:.3f}, {coor1[1]/10:.3f});
    """)
    ax.plot([coor0[0], coor1[0]], [coor0[1], coor1[1]], color=(r, g, b), linewidth=1)

tex_lines.append(r"\end{tikzpicture}")

# Write the file efficiently
with open(f"tikz_image_web_{ecc}.tex", "w") as f:
    f.write("\n".join(tex_lines))

print("tikz_image_colormap.tex generated successfully.")
plt.savefig(f"tikz_image_web_{ecc}.png", dpi=300, bbox_inches='tight')
f_min*3, f_max*3

tikz_image_colormap.tex generated successfully.


(0.0007647067941128484, 0.5028963037191561)

In [17]:
print_order = ['Frame_ring_keys', 'Radials_keys + connections', 'Spiral_center_keys', 'Spiral_catching_keys + UTurn_keys']
section_names = ['Frame_ring', 'Spiral_center', 'Frame_sec', 'Radials_keys', 'Spiral_catching', 'UTurn_keys']
All_sections_nonuniform = [net.Frame_ring_keys, net.Spiral_center_keys, net.Frame_sec_keys, net.Radials_keys, net.Spiral_catching_keys, net.UTurn_keys]

import matplotlib.pyplot as plt
netf = Network_custom()
tex_lines = [r"\begin{tikzpicture}"]

net.direction = {}
for section in All_sections_nonuniform:
    counter = 0
    for key, edge_key in reversed(section.items()):
        if edge_key in section_names:
            continue
        counter += 1
        edge = net.edges[edge_key]
        v0, v1 = edge
        R = net.R[edge_key]
        theta = net.th[edge_key]

        if counter %2 == 0:
            direction = 1
            net.direction[edge_key] = 1
        else:
            direction = -1
            net.direction[edge_key] = -1

        coor0, coor1 = net.vertex_xyz(v0), net.vertex_xyz(v1)
        center, theta0, theta1 = netf._points_on_arc(coor0[:2], coor1[:2], R, print_parameters = True, dir=direction)
        if R/10 > 10000:
            tex_lines.append(fr"""
            \draw[line width=0.5pt, color=black]
            
                ({coor0[0]/10:.3f}, {coor0[1]/10:.3f}) -- 
                ({coor1[0]/10:.3f}, {coor1[1]/10:.3f});
            """)
        elif theta0 > theta1:
            tex_lines.append(fr"""
            \draw ({coor0[0]/10:.3f}, {coor0[1]/10:.3f}) arc ({theta0*180/np.pi:.3f}:{theta1*180/np.pi:.3f}:{R/10:.3f});""")
        else:
            tex_lines.append(fr"""
            \draw ({coor1[0]/10:.3f}, {coor1[1]/10:.3f}) arc ({theta1*180/np.pi:.3f}:{theta0*180/np.pi:.3f}:{R/10:.3f});""")

tex_lines.append(r"\end{tikzpicture}")

# Write the file efficiently
with open("tikz_image_web_arc.tex", "w") as f:
    f.write("\n".join(tex_lines))

print("tikz_image_colormap.tex generated successfully.")


tikz_image_colormap.tex generated successfully.


In [ ]:
file_name_gcode = "Web_constant_length_ecc2_new.gcode"
gcode_path = os.path.join(spider_root, "data", "gcode", file_name_gcode)

vertices_g = netf.read_gcode(gcode_path, save_every_n=2)

In [ ]:
tikz_text = netf.tikz_string_vertices(vertices_g, scale=10)
with open("tikz_image_web_arc2.tex", "w") as f:
    f.write("\n".join(tikz_text))

In [ ]:
tikz_text